In [1]:
#pip install gmaps

In [2]:
# Import Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
from config import gkey
import gmaps
import numpy as np
from scipy.stats import linregress
from matplotlib import pyplot as plt

In [3]:
# Import the previously-created CSV file
weather_df = pd.read_csv("weather_data.csv")

# Reduce the number of cities so we can test
#weather_df = weather_df[:50]
weather_df.head()

,City Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Vaini,-21.2000,-175.2000,77.16,83,75,10.36
1,Lüderitz,-26.6481,15.1594,66.85,33,0,12.33
2,San Juan,-31.5375,-68.5364,64.72,59,0,4.85
3,Pontes e Lacerda,-15.2261,-59.3353,79.12,48,99,2.46
4,Lebu,-37.6167,-73.6500,47.19,83,96,6.87


In [4]:
# Configure GMAPS with API key
gmaps.configure(api_key = gkey)

# Store latitude and longitude into locations
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Store humidity values as well
humidities = weather_df['Humidity'].astype(float)

In [5]:
# Create a heatmap layer
fig = gmaps.figure(center=(0, 0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=humidities, dissipating=False, 
                                 max_intensity=max(weather_df['Humidity']), point_radius = 4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Narrow down the weather_df to match my ideal conditions
max_temp = 60
min_temp = 25
min_humidity = 10
max_humidity = 50
max_windspeed = 10
min_cloudiness = 0
max_cloudiness = 50

# Drop values of non-ideal locations
index_to_drop = weather_df[
    (weather_df["Temperature"] > max_temp) |
    (weather_df["Temperature"] < min_temp) |
    (weather_df["Humidity"] > max_humidity) |
    (weather_df["Humidity"] < min_humidity) |
    (weather_df["Cloudiness"] > max_cloudiness) |
    (weather_df["Cloudiness"] < min_cloudiness) |
    (weather_df["Wind Speed"] > max_windspeed)
].index

weather_df.drop(index_to_drop , inplace=True)

# View reduced dataframe
weather_df
#print(f"There are {len(weather_df)} hotels")

,City Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
67,Bredasdorp,-34.5322,20.0403,56.52,47,6,8.41
189,Aklavik,68.2191,-135.0107,47.23,37,12,9.01
259,Kruisfontein,-34.0033,24.7314,54.70,27,0,8.25
266,Sanandij,35.3144,46.9923,59.04,32,0,1.72
276,Port Elizabeth,-33.9180,25.5701,57.49,41,0,9.22
327,Makīnsk,52.6292,70.4169,59.97,41,4,1.07
384,Leh,34.1667,77.5833,51.35,24,0,4.34


In [7]:
# Find the closest hotel to each set of coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "lodging",
    "radius": 5000,
    "key": gkey
}

# Hotels array
hotels = []

# Loop through the dataframe
for index, row in weather_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    location = f"{lat},{lng}"
    params["location"] = location
    
    hotel_data = requests.get(base_url, params=params).json()

    try:
        hotels.append(hotel_data["results"][0]["name"])
        
    except:
        hotels.append("n/a")

# Assign new column to existing df
weather_df["Hotel"] = hotels

In [8]:
# Create a map using state centroid coordinates to set markers
marker_locations = weather_df[['Latitude', 'Longitude']]
info_content = []

for index, row in weather_df.iterrows():
    name = row["Hotel"]
    city = row["City Name"]
    string = f"{name} hotel, {city}"
    info_content.append(string)

for x in info_content:
    print(f"{x}") 

Bredasdorp Country Manor hotel, Bredasdorp
Aklavik Inn hotel, Aklavik
Oyster Bay House Rental hotel, Kruisfontein
Sanandaj Tourist Hotel hotel, Sanandij
39 On Nile Guest House hotel, Port Elizabeth
Bereke hotel, Makīnsk
Hotel Ladakh Greens hotel, Leh


In [11]:
# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure(center=(0, 0), zoom_level=1)
markers = gmaps.marker_layer(marker_locations, info_box_content=[f"{x}" for x in info_content])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))